test: 
1. 1 fac_type (max_q<cap<total_q)
2. min cost 
3. 
4. potential location: top 30 production

In [1]:
# define the optimisation problem

from pulp import *

prob = LpProblem('prob', LpMinimize)

In [2]:
# define parameters

import pandas as pd
import numpy as np

# sources and production

df = pd.read_excel('/Users/kouchenlu/Desktop/Transport Strategy.xlsx','Weighted Production')
s = df['Source'].tolist()                        #list of sources of products
q = df['Weighted Production'].tolist()           #weighted production of each sorce

S = [('S'+str(k+1)) for k in range(len(s))]
Q = {source:(q[k]) for k,source in enumerate(S)}

Q

{'S1': 0.23671836673082,
 'S2': 1.495490395863995,
 'S3': 0.8709234310728706,
 'S4': 8.060402279331296,
 'S5': 0.01963092946903656,
 'S6': 0.0878448243329937,
 'S7': 0.8331710630589833,
 'S8': 0.3382693686554211,
 'S9': 0.1004641684591872,
 'S10': 0.006928563342012907,
 'S11': 0.240227933129552,
 'S12': 0.01449866032680478,
 'S13': 0.6477275812849764,
 'S14': 0.0409373938639194,
 'S15': 1.610204158647496,
 'S16': 0.0008528623721649874,
 'S17': 0.1921053624665082,
 'S18': 1.123219744141288,
 'S19': 0.01876297218762972,
 'S20': 0.3162157062530661,
 'S21': 0.07846333823917882,
 'S22': 0.008279557719159215,
 'S23': 1.561436040871686,
 'S24': 0.3406543643156345,
 'S25': 0.005970036605154913,
 'S26': 0.04747348956564398,
 'S27': 0.01876297218762972,
 'S28': 0.002558587116494962,
 'S29': 0.1466545907392732,
 'S30': 0.3158005962489151,
 'S31': 0.04434884335257935,
 'S32': 0.02473300879278463,
 'S33': 0.6991358164459036,
 'S34': 0.02453677497264048,
 'S35': 0.1466923280123778,
 'S36': 0.0221744

In [3]:
# location and type of factories

df = pd.read_excel('/Users/kouchenlu/Desktop/Transport Strategy.xlsx','Town-test')
t = df['Town'].tolist()  

T = [('T'+str(i+1)) for i in range(len(t))]      #list of potential location of factories

T

['T1',
 'T2',
 'T3',
 'T4',
 'T5',
 'T6',
 'T7',
 'T8',
 'T9',
 'T10',
 'T11',
 'T12',
 'T13',
 'T14',
 'T15',
 'T16',
 'T17',
 'T18',
 'T19',
 'T20',
 'T21',
 'T22',
 'T23',
 'T24',
 'T25',
 'T26',
 'T27',
 'T28',
 'T29',
 'T30']

In [4]:
df = pd.read_excel('/Users/kouchenlu/Desktop/Transport Strategy.xlsx','Factory Type-test2')
f = df['Fac_Type'].tolist()
cap_fac = df['Capacity'].tolist()
c_fac = df['Cost'].tolist()

F = [('F'+str(j+1)) for j in range(len(f))]   #set of factory types
Cap_F = {factype:(cap_fac[j]) for j,factype in enumerate(F)}
C_F = {factype:(c_fac[j]) for j,factype in enumerate(F)}

Cap_F
C_F

{'F1': 4745000}

In [5]:
# export sites

df = pd.read_excel('/Users/kouchenlu/Desktop/Transport Strategy.xlsx','Export Site')

p = df['Location'].tolist()
E = [('E'+str(l+1)) for l in range(len(p))]    #set of export sites

E

['E1', 'E2']

In [6]:
# modes

df = pd.read_excel('/Users/kouchenlu/Desktop/Transport Strategy.xlsx','Mode 1')

mode1 = df['Mode'].tolist()
M = [('M'+str(m+1)) for m in range(len(mode1))]     #set of modes from sources to factories

c_m1_int = df['Initial Cost'].tolist()
c_m1_ship = df['Shippment Cost'].tolist()
c_m1_cat = df['Category'].tolist()
c_m1_cap = df['Capacity'].tolist()
C_M_INT_W = {mode:(c_m1_int[m]*1000/c_m1_cap[m]) for m,mode in enumerate(M)}
C_M_SHIP = {mode:(c_m1_ship[m]) for m,mode in enumerate(M)}


v_m1 = df['Speed'].tolist()

df = pd.read_excel('/Users/kouchenlu/Desktop/Transport Strategy.xlsx','Mode 2')

mode2 = df['Mode'].tolist()
N = [('N'+str(n+1)) for n in range(len(mode2))]     #set of modes from sources to factories

c_m2_int = df['Initial Cost'].tolist()
c_m2_ship = df['Shippment Cost'].tolist()
c_m2_cat = df['Category'].tolist()
c_m2_cap = df['Capacity'].tolist()
C_N_INT_W = {mode:(c_m2_int[n]*1000/c_m2_cap[n]) for n,mode in enumerate(N)}
C_N_SHIP = {mode:(c_m2_ship[n]) for n,mode in enumerate(N)}

v_m2 = df['Speed'].tolist()

C_N_SHIP

{'N1': 0.125,
 'N2': 0.04349609156548648,
 'N3': 0.04349609156548648,
 'N4': 0.03,
 'N5': 3.0,
 'N6': 1.75,
 'N7': 0.02,
 'N8': 0.035}

In [7]:
dis_sf_water = pd.read_excel('/Users/kouchenlu/Desktop/Transport Strategy.xlsx','OD Matrix-SF-Water').iloc[0:, 1:].values.tolist()
dis_sf_road = pd.read_excel('/Users/kouchenlu/Desktop/Transport Strategy.xlsx','OD Matrix-SF-Road').iloc[0:, 1:].values.tolist()

dis_sf = np.zeros((len(s), len(t), len(f), len(mode1)))

print(len(s))
print(len(t))
print(len(f))
print(len(mode1))

print(len(dis_sf_water))
print(len(dis_sf_water[0]))
print(len(dis_sf_road))
print(len(dis_sf_road[0]))

print(len(dis_sf))
print(len(dis_sf[0]))
print(len(dis_sf[0][0]))
print(len(dis_sf[0][0][0]))


340
30
1
4
340
767
340
767
340
30
1
4


In [8]:
for k in range(len(s)):
    for i in range(len(t)):
        for j in range(len(f)):
            for m in range (len(mode1)):
                if c_m1_cat[m] == 'Water':
                    dis_sf[k][i][j][m] = dis_sf_water[k][i]
                if c_m1_cat[m] == 'Road':
                    dis_sf[k][i][j][m] = dis_sf_road[k][i]
                    

T_SF = {
    source: {
        town: {
            factype: {
                mode: dis_sf[k][i][j][m]/(1000*v_m1[m]) 
                for m, mode in enumerate(M)
            } 
            for j, factype in enumerate(F)
        } 
        for i, town in enumerate(T)
    } 
    for k, source in enumerate(S)
}

print(f"T_SF: {str(T_SF)[:1000]}")

D_SF = {
    source: {
        town: {
            factype: {
                mode: dis_sf[k][i][j][m]/1000 
                for m, mode in enumerate(M)
            } 
            for j, factype in enumerate(F)
        } 
        for i, town in enumerate(T)
    } 
    for k, source in enumerate(S)
}


T_SF: {'S1': {'T1': {'F1': {'M1': 0.0, 'M2': 0.0, 'M3': 0.0, 'M4': 0.0}}, 'T2': {'F1': {'M1': 51.30938142732, 'M2': 48.866077549828574, 'M3': 124999.9999875, 'M4': 124999.9999875}}, 'T3': {'F1': {'M1': 102.48972913744001, 'M2': 97.60926584518096, 'M3': 124999.9999875, 'M4': 124999.9999875}}, 'T4': {'F1': {'M1': 499999.99995, 'M2': 476190.47614285717, 'M3': 124999.9999875, 'M4': 124999.9999875}}, 'T5': {'F1': {'M1': 145.74219170200502, 'M2': 138.80208733524287, 'M3': 124999.9999875, 'M4': 124999.9999875}}, 'T6': {'F1': {'M1': 153.58190194856002, 'M2': 146.26847804624762, 'M3': 124999.9999875, 'M4': 124999.9999875}}, 'T7': {'F1': {'M1': 17.19809998784, 'M2': 16.379142845561905, 'M3': 124999.9999875, 'M4': 124999.9999875}}, 'T8': {'F1': {'M1': 97.341876474765, 'M2': 92.70654902358572, 'M3': 124999.9999875, 'M4': 124999.9999875}}, 'T9': {'F1': {'M1': 499999.99995, 'M2': 476190.47614285717, 'M3': 124999.9999875, 'M4': 124999.9999875}}, 'T10': {'F1': {'M1': 47.02582943342, 'M2': 44.786504222

In [9]:
dis_fe_water = pd.read_excel('/Users/kouchenlu/Desktop/Transport Strategy.xlsx','OD Matrix-FE-Water').iloc[0:, 1:].values.tolist()
dis_fe_road = pd.read_excel('/Users/kouchenlu/Desktop/Transport Strategy.xlsx','OD Matrix-FE-Road').iloc[0:, 1:].values.tolist()
#dis_fe_air = pd.read_excel('/Users/kouchenlu/Desktop/Transport Strategy.xlsx','OD Matrix-FE-Air').iloc[0:, 1:].values.tolist()
dis_fe_rail = pd.read_excel('/Users/kouchenlu/Desktop/Transport Strategy.xlsx','OD Matrix-FE-Rail').iloc[0:, 1:].values.tolist()

dis_fe = np.zeros((len(t), len(f), len(p), len(mode2)))

for i in range(len(t)):
    for j in range(len(f)):
        for l in range(len(p)):
            for n in range (len(mode2)):
                if c_m2_cat[n] == 'Water':
                    dis_fe[i][j][l][n] = dis_fe_water[i][l]
                if c_m2_cat[n] == 'Road':
                    dis_fe[i][j][l][n] = dis_fe_road[i][l]
                    #dis_fe[i][j][l][n] = 9999999999
                if c_m2_cat[n] == 'Air':
                    #dis_fe[i][j][l][n] = dis_fe_air[i][l]
                    dis_fe[i][j][l][n] = 9999999999
                if c_m2_cat[n] == 'Rail':
                    dis_fe[i][j][l][n] = dis_fe_rail[i][l]

T_FE = {
    town: {
        factype: {
            export: {
                mode: dis_fe[i][j][l][n]/(1000*v_m2[n]) 
                for n, mode in enumerate(N)
            } 
            for l, export in enumerate(E)
        } 
        for j, factype in enumerate(F)
    } 
    for i, town in enumerate(T)
}

print(f"T_FE: {str(T_FE)[:1000]}")

D_FE = {
    town: {
        factype: {
            export: {
                mode: dis_fe[i][j][l][n]/1000 
                for n, mode in enumerate(N)
            } 
            for l, export in enumerate(E)
        } 
        for j, factype in enumerate(F)
    } 
    for i, town in enumerate(T)
}

T_FE: {'T1': {'F1': {'E1': {'N1': 82.71628595761, 'N2': 78.77741519772381, 'N3': 78.77741519772381, 'N4': 137.86047659601667, 'N5': 124999.9999875, 'N6': 124999.9999875, 'N7': 166666.66665, 'N8': 166666.66665}, 'E2': {'N1': 499999.99995, 'N2': 476190.47614285717, 'N3': 476190.47614285717, 'N4': 833333.33325, 'N5': 124999.9999875, 'N6': 124999.9999875, 'N7': 166666.66665, 'N8': 166666.66665}}}, 'T2': {'F1': {'E1': {'N1': 106.44513371625, 'N2': 101.376317825, 'N3': 101.376317825, 'N4': 177.40855619375, 'N5': 124999.9999875, 'N6': 124999.9999875, 'N7': 166666.66665, 'N8': 166666.66665}, 'E2': {'N1': 75.312497587325, 'N2': 71.72618817840477, 'N3': 71.72618817840477, 'N4': 125.52082931220833, 'N5': 8.000725, 'N6': 8.000725, 'N7': 10.667633333333333, 'N8': 10.667633333333333}}}, 'T3': {'F1': {'E1': {'N1': 135.16323625436, 'N2': 128.72689167081904, 'N3': 128.72689167081904, 'N4': 225.2720604239333, 'N5': 124999.9999875, 'N6': 124999.9999875, 'N7': 166666.66665, 'N8': 166666.66665}, 'E2': {'N1

In [10]:
# define decision varibles

x = LpVariable.dicts('x', (T,F), lowBound = 0, upBound = 1, cat = LpInteger)      # is there a factory of type j in town i
y = LpVariable.dicts('y', (S,T,F,M), lowBound = 0, upBound = 1, cat = LpInteger)  # whether products collected from source k are transported to a type j factory in town i by mode m
z = LpVariable.dicts('z', (T,F,E,N), lowBound = 0, upBound = 1, cat = LpInteger)  # whether products processed in a type j factory in town i are transported to an export site l by mode n

In [11]:
# define the objective function

# min cost - construction cost and transport cost for 1 ton
prob += lpSum([x[i][j] * C_F[j] for j in F for i in T ]) +\
        lpSum([y[k][i][j][m] * D_SF[k][i][j][m] * C_M_SHIP[m] + C_M_INT_W[m] for k in S for i in T for j in F for m in M]) +\
        lpSum([z[i][j][l][n] * D_FE[i][j][l][n] * C_N_SHIP[n] + C_N_INT_W[n] for i in T for j in F for l in E for n in N])

In [12]:
# define constraints

# factory construction - no more than 1 factory in a town
for i in T:
    prob += lpSum([x[i][j] for j in F]) <= 1

# source and factory distribution - it is possible for y to = 1 only if x = 1 -- change
for i in T:
    for j in F:
        for k in S:
            for m in M:
                prob += y[k][i][j][m] <= x[i][j]

# factory and export site distribution - it is possible for z to = 1 only if x = 1 -- change
for i in T:
    for j in F:
        for l in E:  
            for n in N:
                prob += z[i][j][l][n] <= x[i][j]
        
# ensure that products from each source are transported to 1 factory using 1 mode -- delete
for k in S:
    prob += lpSum([y[k][i][j][m] for j in F for i in T for m in M]) == 1


# ensure that products from each factory are transported to 1 export site using 1 mode -- delete and add * x[i][j]
for i in T:
    for j in F:
        prob += lpSum([z[i][j][l][n] for l in E for n in N]) == 1 * x[i][j]

# factory capacity constraint
for i in T:
    for j in F:
        prob += lpSum(y[k][i][j][m] * Q[k] for k in S for m in M) <= x[i][j] * Cap_F[j]


In [13]:
# solution

status = prob.solve()

print("STATUS\n" + LpStatus[status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kouchenlu/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ww/3ktk25615vv9klx14jsbd9j80000gn/T/b8b346f44b0e488391fc56978dffc968-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/ww/3ktk25615vv9klx14jsbd9j80000gn/T/b8b346f44b0e488391fc56978dffc968-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 41715 COLUMNS
At line 330208 RHS
At line 371919 BOUNDS
At line 413230 ENDATA
Problem MODEL has 41710 rows, 41310 columns and 164610 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 8.22416e+06 - 23.86 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 480 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 450 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 420 strengthened rows, 0 substitution

Cbc0010I After 4400 nodes, 705 on tree, 9506022.9 best solution, best possible 8224275.1 (777.90 seconds)
Cbc0010I After 4500 nodes, 681 on tree, 9506022.9 best solution, best possible 8224275.1 (791.19 seconds)
Cbc0010I After 4600 nodes, 675 on tree, 9506022.9 best solution, best possible 8224275.1 (805.30 seconds)
Cbc0010I After 4700 nodes, 647 on tree, 9506022.9 best solution, best possible 8224275.1 (819.32 seconds)
Cbc0038I Full problem 41440 rows 41100 columns, reduced to 789 rows 788 columns
Cbc0010I After 4800 nodes, 668 on tree, 9506022.9 best solution, best possible 8224275.1 (833.87 seconds)
Cbc0010I After 4900 nodes, 658 on tree, 9506022.9 best solution, best possible 8224275.1 (849.39 seconds)
Cbc0010I After 5000 nodes, 637 on tree, 9506022.9 best solution, best possible 8224275.1 (862.74 seconds)
Cbc0010I After 5100 nodes, 679 on tree, 9506022.9 best solution, best possible 8224275.1 (879.03 seconds)
Cbc0010I After 5200 nodes, 662 on tree, 9506022.9 best solution, best po

Cbc0010I After 19000 nodes, 222 on tree, 9506022.9 best solution, best possible 8237550.5 (8316.81 seconds)
Cbc0010I After 19100 nodes, 222 on tree, 9506022.9 best solution, best possible 8237631.3 (8338.54 seconds)
Cbc0010I After 19200 nodes, 222 on tree, 9506022.9 best solution, best possible 8237687.1 (8364.04 seconds)
Cbc0010I After 19300 nodes, 222 on tree, 9506022.9 best solution, best possible 8237753.4 (8388.42 seconds)
Cbc0010I After 19400 nodes, 220 on tree, 9506022.9 best solution, best possible 8237802.3 (8412.51 seconds)
Cbc0010I After 19500 nodes, 214 on tree, 9506022.9 best solution, best possible 8237862.3 (8438.77 seconds)
Cbc0010I After 19600 nodes, 214 on tree, 9506022.9 best solution, best possible 8237909.3 (8464.50 seconds)
Cbc0010I After 19700 nodes, 214 on tree, 9506022.9 best solution, best possible 8237961.8 (8486.79 seconds)
Cbc0010I After 19800 nodes, 214 on tree, 9506022.9 best solution, best possible 8238004.5 (8510.86 seconds)
Cbc0010I After 19900 nodes, 

Cbc0010I After 34000 nodes, 327 on tree, 9506022.9 best solution, best possible 8244897.5 (10655.45 seconds)
Cbc0010I After 34100 nodes, 326 on tree, 9506022.9 best solution, best possible 8244940.3 (10665.37 seconds)
Cbc0010I After 34200 nodes, 326 on tree, 9506022.9 best solution, best possible 8244974.5 (10675.89 seconds)
Cbc0010I After 34300 nodes, 328 on tree, 9506022.9 best solution, best possible 8245016.4 (10685.63 seconds)
Cbc0010I After 34400 nodes, 328 on tree, 9506022.9 best solution, best possible 8245065.8 (10695.97 seconds)
Cbc0010I After 34500 nodes, 329 on tree, 9506022.9 best solution, best possible 8245105.7 (10706.23 seconds)
Cbc0010I After 34600 nodes, 330 on tree, 9506022.9 best solution, best possible 8245155.1 (10717.40 seconds)
Cbc0010I After 34700 nodes, 333 on tree, 9506022.9 best solution, best possible 8245198.3 (10729.15 seconds)
Cbc0010I After 34800 nodes, 335 on tree, 9506022.9 best solution, best possible 8245261.4 (10798.62 seconds)
Cbc0010I After 3490

Cbc0010I After 48800 nodes, 1072 on tree, 9506022.9 best solution, best possible 8249926.1 (12087.54 seconds)
Cbc0010I After 48900 nodes, 1077 on tree, 9506022.9 best solution, best possible 8249930.8 (12094.42 seconds)
Cbc0010I After 49000 nodes, 1084 on tree, 9506022.9 best solution, best possible 8249937.9 (12101.66 seconds)
Cbc0010I After 49100 nodes, 1081 on tree, 9506022.9 best solution, best possible 8249937.9 (12108.12 seconds)
Cbc0010I After 49200 nodes, 1084 on tree, 9506022.9 best solution, best possible 8249937.9 (12116.87 seconds)
Cbc0010I After 49300 nodes, 1073 on tree, 9506022.9 best solution, best possible 8249937.9 (12123.42 seconds)
Cbc0010I After 49400 nodes, 1086 on tree, 9506022.9 best solution, best possible 8249937.9 (12129.04 seconds)
Cbc0010I After 49500 nodes, 1075 on tree, 9506022.9 best solution, best possible 8249937.9 (12137.88 seconds)
Cbc0010I After 49600 nodes, 1078 on tree, 9506022.9 best solution, best possible 8249937.9 (12148.06 seconds)
Cbc0010I A

...

Cbc0010I After 182100 nodes, 2245 on tree, 9506022.9 best solution, best possible 8256775.6 (20855.85 seconds)
Cbc0010I After 182200 nodes, 2265 on tree, 9506022.9 best solution, best possible 8256777.5 (20860.05 seconds)
Cbc0010I After 182300 nodes, 2281 on tree, 9506022.9 best solution, best possible 8256779 (20864.09 seconds)
Cbc0010I After 182400 nodes, 2302 on tree, 9506022.9 best solution, best possible 8256780.5 (20868.02 seconds)
Cbc0010I After 182500 nodes, 2321 on tree, 9506022.9 best solution, best possible 8256782.3 (20872.67 seconds)
Cbc0010I After 182600 nodes, 2337 on tree, 9506022.9 best solution, best possible 8256784.1 (20876.64 seconds)
Cbc0010I After 182700 nodes, 2354 on tree, 9506022.9 best solution, best possible 8256785.6 (20880.82 seconds)
Cbc0010I After 182800 nodes, 2360 on tree, 9506022.9 best solution, best possible 8256787.2 (20885.09 seconds)
Cbc0010I After 182900 nodes, 2372 on tree, 9506022.9 best solution, best possible 8256789.1 (20889.27 seconds)
Cbc

Cbc0010I After 196800 nodes, 1938 on tree, 9506022.9 best solution, best possible 8257069.9 (21542.64 seconds)
Cbc0010I After 196900 nodes, 1930 on tree, 9506022.9 best solution, best possible 8257074.3 (21547.96 seconds)
Cbc0010I After 197000 nodes, 1930 on tree, 9506022.9 best solution, best possible 8257078.3 (21553.36 seconds)
Cbc0010I After 197100 nodes, 1900 on tree, 9506022.9 best solution, best possible 8257078.3 (21557.40 seconds)
Cbc0010I After 197200 nodes, 1864 on tree, 9506022.9 best solution, best possible 8257078.3 (21560.87 seconds)
Cbc0010I After 197300 nodes, 1848 on tree, 9506022.9 best solution, best possible 8257078.3 (21565.56 seconds)
Cbc0010I After 197400 nodes, 1837 on tree, 9506022.9 best solution, best possible 8257078.3 (21570.46 seconds)
Cbc0010I After 197500 nodes, 1822 on tree, 9506022.9 best solution, best possible 8257078.3 (21575.24 seconds)
Cbc0010I After 197600 nodes, 1802 on tree, 9506022.9 best solution, best possible 8257078.3 (21580.52 seconds)
C

Cbc0010I After 211600 nodes, 680 on tree, 9506022.9 best solution, best possible 8258083.7 (22287.94 seconds)
Cbc0010I After 211700 nodes, 671 on tree, 9506022.9 best solution, best possible 8258101.8 (22293.30 seconds)
Cbc0010I After 211800 nodes, 660 on tree, 9506022.9 best solution, best possible 8258120.6 (22298.34 seconds)
Cbc0010I After 211900 nodes, 652 on tree, 9506022.9 best solution, best possible 8258137.2 (22303.75 seconds)
Cbc0010I After 212000 nodes, 645 on tree, 9506022.9 best solution, best possible 8258150.3 (22308.92 seconds)
Cbc0010I After 212100 nodes, 646 on tree, 9506022.9 best solution, best possible 8258162.1 (22314.31 seconds)
Cbc0010I After 212200 nodes, 638 on tree, 9506022.9 best solution, best possible 8258172.4 (22319.68 seconds)
Cbc0010I After 212300 nodes, 638 on tree, 9506022.9 best solution, best possible 8258183.9 (22325.14 seconds)
Cbc0010I After 212400 nodes, 638 on tree, 9506022.9 best solution, best possible 8258194.1 (22330.65 seconds)
Cbc0010I A

Cbc0010I After 226400 nodes, 685 on tree, 9506022.9 best solution, best possible 8259579.5 (22988.89 seconds)
Cbc0010I After 226500 nodes, 708 on tree, 9506022.9 best solution, best possible 8259583.2 (22993.03 seconds)
Cbc0010I After 226600 nodes, 729 on tree, 9506022.9 best solution, best possible 8259589.6 (22996.81 seconds)
Cbc0010I After 226700 nodes, 755 on tree, 9506022.9 best solution, best possible 8259594.9 (23000.70 seconds)
Cbc0010I After 226800 nodes, 770 on tree, 9506022.9 best solution, best possible 8259603.4 (23004.84 seconds)
Cbc0010I After 226900 nodes, 775 on tree, 9506022.9 best solution, best possible 8259611.8 (23009.46 seconds)
Cbc0010I After 227000 nodes, 780 on tree, 9506022.9 best solution, best possible 8259619.1 (23014.34 seconds)
Cbc0010I After 227100 nodes, 798 on tree, 9506022.9 best solution, best possible 8259625.1 (23018.49 seconds)
Cbc0010I After 227200 nodes, 810 on tree, 9506022.9 best solution, best possible 8259631.1 (23023.28 seconds)
Cbc0010I A

STATUS
Optimal


In [14]:
import math

total_fac = 0

# Output values of x
print("Location of factories:")
for i in T:
    for j in F:
        if x[i][j].varValue == 1:
            index_i = int(i[1:]) - 1
            index_j = int(j[1:]) - 1
            total_fac += 1
            print(f"Factory {f[index_j]} located in: {t[index_i]}")
            #print(f"{t[index_i]}")

print(f"Total number of factories: {total_fac}")
            
# Output values of y
print("\nFactory assignment:")

output_s =[]
output_s_q = []
output_fac_sf = []
output_mode1 = []
output_mode1_n = []
output_dis_sf = []
output_cost_sf = []
output_time_sf = []

for k in S:
    for i in T:
        for j in F:
            for m in M:
                if y[k][i][j][m].varValue == 1:
                    index_k = int(k[1:]) - 1
                    index_i = int(i[1:]) - 1
                    index_j = int(j[1:]) - 1
                    index_m = int(m[1:]) - 1
                    print(f"Source {s[index_k]} --> {mode1[index_m]} --> Factory {t[index_i]} -- {f[index_j]}")
                    #wirte to excel
                    output_s.append(s[index_k])
                    output_s_q.append(q[index_k])
                    output_fac_sf.append(t[index_i])
                    output_mode1.append(mode1[index_m])
                    output_mode1_n.append(math.ceil(q[index_k]/c_m1_cap[index_m]))
                    output_dis_sf.append(dis_sf[index_k][index_i][index_j][index_m])
                    output_cost_sf.append(math.ceil(q[index_k]/c_m1_cap[index_m])*c_m1_int[index_m] + q[index_k]*dis_sf[index_k][index_i][index_j][index_m]/1000*c_m1_ship[index_m])
                    output_time_sf.append(dis_sf[index_k][index_i][index_j][index_m]/(1000*v_m1[index_m]))

# Output values of z
print("\nPort assignment:")

output_fac_fe = []
output_p = []
output_mode2 = []
output_mode2_cap = []
output_mode2_n = []
output_mode2_int = []
output_mode2_ship = []
output_dis_fe = []
output_cost_fe = []
output_time_fe = []
                                                                                      
for i in T:
    for j in F:
        for l in E:
            for n in N:
                if z[i][j][l][n].varValue == 1:
                    index_i = int(i[1:]) - 1
                    index_j = int(j[1:]) - 1
                    index_l = int(l[1:]) - 1
                    index_n = int(n[1:]) - 1
                    print(f"Source {p[index_l]} <-- {mode2[index_n]} <-- Factory {t[index_i]} -- {f[index_j]}")
                    #wirte to excel
                    output_fac_fe.append(t[index_i])
                    output_p.append(p[index_l])
                    output_mode2.append(mode2[index_n])
                    output_mode2_cap.append(c_m2_cap[index_n])
                    output_mode2_int.append(c_m2_int[index_n])
                    output_mode2_ship.append(c_m2_ship[index_n])
                    output_mode2_n.append(0)
                    output_dis_fe.append(dis_fe[index_i][index_j][index_l][index_n])
                    output_cost_fe.append(0)
                    output_time_fe.append(dis_fe[index_i][index_j][index_l][index_n]/(1000*v_m2[index_n]))
                   

Location of factories:
Factory Superfrio located in: Humaitá
Factory Superfrio located in: Boca do Acre
Total number of factories: 2

Factory assignment:
Source Abaetetuba --> Mid Size ferry --> Factory Humaitá -- Superfrio
Source Acará --> Mid Size ferry --> Factory Humaitá -- Superfrio
Source Acrelândia --> Mid Size ferry --> Factory Boca do Acre -- Superfrio
Source Afuá --> Ferry with Freezer --> Factory Humaitá -- Superfrio
Source Água Azul do Norte --> Mid Size ferry --> Factory Humaitá -- Superfrio
Source Alcântara --> Mid Size ferry --> Factory Humaitá -- Superfrio
Source Alenquer --> Mid Size ferry --> Factory Humaitá -- Superfrio
Source Almeirim --> Mid Size ferry --> Factory Humaitá -- Superfrio
Source Alta Floresta --> Mid Size ferry --> Factory Humaitá -- Superfrio
Source Alta Floresta D'Oeste --> Mid Size ferry --> Factory Boca do Acre -- Superfrio
Source Altamira --> Mid Size ferry --> Factory Humaitá -- Superfrio
Source Alto Alegre do Pindaré --> Refrigerator truck (Merc

In [15]:
# Output of total cost
#print("\nTotal cost:")
#print(f"{prob.objective.value()} dollars")

# Output of total time
print("\nTotal time:")

total_time = 0

for k in S:
    for i in T:
        for j in F:
            for m in M:
                total_time += y[k][i][j][m].varValue * T_SF[k][i][j][m]
                
for i in T:
    for j in F:
        for l in E:
            for n in N:
                total_time += z[i][j][l][n].varValue * T_FE[i][j][l][n]
                
print(f"{total_time} hours")


Total time:
17565.07834005845 hours


In [16]:
# Output

df_sf = pd.DataFrame({'Source': output_s, 'Production': output_s_q, 'Factory': output_fac_sf, 'Mode': output_mode1, 'Num of Veh': output_mode1_n, 
                  'Distance': output_dis_sf, 'Time': output_time_sf, 'Transport Cost': output_cost_sf})

df_fe = pd.DataFrame({'Factory': output_fac_fe, 'Port': output_p, 'Mode': output_mode2, 'Capacity': output_mode2_cap, 'Num of Veh': output_mode2_n, 
                  'Distance': output_dis_fe, 'Time': output_time_fe, 'Initial Cost': output_mode2_int, 'Shippment Cost': output_mode2_ship, 'Transport Cost': output_cost_fe})

filename = 'Output_Min_Cost_Final_new.xlsx'

sheet_name_sf = 'Sheet_SF'
sheet_name_fe = 'Sheet_FE'

with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    df_sf.to_excel(writer, sheet_name=sheet_name_sf, index=False)  
    df_fe.to_excel(writer, sheet_name=sheet_name_fe, index=False)  